In [19]:
import setuptools.dist
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam 
# from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers.schedules import ExponentialDecay 
from keras.models import model_from_json 
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd

In [6]:
df = pd.read_csv('fer2013.csv')
print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [7]:
X_train = []
y_train = []
X_test = []
y_test = []
for index, row in df.iterrows():
    k = row['pixels'].split(" ")
    if row['Usage'] == 'Training':
        X_train.append(np.array(k))
        y_train.append(row['emotion'])
    elif row['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_test.append(row['emotion'])

In [8]:
X_train = np.array(X_train, dtype = 'uint8')
y_train = np.array(y_train, dtype = 'uint8')
X_test = np.array(X_test, dtype = 'uint8')
y_test = np.array(y_test, dtype = 'uint8')

In [9]:
import keras
from keras.utils import to_categorical
y_train= to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [10]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [12]:
datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode = 'nearest')
testgen = ImageDataGenerator(rescale=1./255)
datagen.fit(X_train)
batch_size = 64

In [13]:
train_flow = datagen.flow(X_train, y_train, batch_size=batch_size) 
test_flow = testgen.flow(X_test, y_test, batch_size=batch_size)

In [21]:
emotion_model = Sequential() 
  
# emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',  
#                          input_shape=(48, 48, 1))) 
# emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu')) 
# emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
# emotion_model.add(Dropout(0.25)) 
  
# emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
# emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
# emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
# emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
# emotion_model.add(Dropout(0.25)) 
  
# emotion_model.add(Flatten()) 
# emotion_model.add(Dense(1024, activation='relu')) 
# emotion_model.add(Dropout(0.5)) 
# emotion_model.add(Dense(7, activation='softmax'))


#Different model
emotion_model = Sequential()

# First convolutional block
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), padding='same'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.3))

# Second convolutional block
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.3))

# Third convolutional block
emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.3))

# Flatten and fully connected layers
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
  
emotion_model.summary() 
  
cv2.ocl.setUseOpenCL(False) 
  
initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=100000,  
                               decay_rate=0.96) 
  
optimizer = Adam(learning_rate=lr_schedule) 
  
emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer,  
                      metrics=['accuracy']) 

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     9,438,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │         7,17

 Total params: 10,593,223 (40.41 MB)

 Trainable params: 10,591,431 (40.40 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [22]:
emotion_model_info = emotion_model.fit( 
        train_flow, 
        steps_per_epoch=int(len(X_train) / batch_size), 
        epochs=100, 
        verbose=1,
        validation_data=test_flow, 
        validation_steps=int(len(X_test) / batch_size)) 

Epoch 1/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 391s 863ms/step - accuracy: 0.2209 - loss: 2.5015 - val_accuracy: 0.2391 - val_loss: 2.4058
Epoch 2/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 192us/step - accuracy: 0.2344 - loss: 1.8035 - val_accuracy: 0.4000 - val_loss: 1.5646
Epoch 3/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 382s 853ms/step - accuracy: 0.2756 - loss: 1.7800 - val_accuracy: 0.3446 - val_loss: 1.6427
Epoch 4/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 83us/step - accuracy: 0.3125 - loss: 1.7824 - val_accuracy: 0.2000 - val_loss: 2.1311
Epoch 5/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 368s 821ms/step - accuracy: 0.3108 - loss: 1.7111 - val_accuracy: 0.3909 - val_loss: 1.5559
Epoch 6/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 74us/step - accuracy: 0.4062 - loss: 1.6455 - val_accuracy: 0.2000 - val_loss: 1.7166
Epoch 7/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 351s 783ms/step - accuracy: 0.3558 - loss: 1.6315 - val_accuracy: 0.4336 - val_loss: 1.4568
Epoch 8/100
448/448 ━━━━━━━━━━━━━━━━━━━━ 1s 76us/step - accuracy: 0.2969 - lo

KeyboardInterrupt: 

In [18]:
emotion_model.evaluate(X_test, y_test)

113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.2857 - loss: 189.6625


[190.7711639404297, 0.2800222933292389]

In [ ]:
model_json = emotion_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
emotion_model.save_weights("model.h5")
print("Saved model to disk")